<a href="https://colab.research.google.com/github/basavarajmullur/Spring-Boot-JdbcTemplate/blob/master/notebooks/quick_start_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install -q transformers accelerate bitsandbytes
!pip install -q fastapi uvicorn pyngrok pillow


## Setup

To complete this tutorial, you'll need to have a runtime with [sufficient resources](https://ai.google.dev/gemma/docs/core#sizes) to run the MedGemma model.

You can try out MedGemma 4B for free in Google Colab using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

**Note**: To run the demo with MedGemma 27B in Google Colab, you will need a runtime with an A100 GPU.

### Get access to MedGemma

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedGemma model page](https://huggingface.co/google/medgemma-1.5-4b-it) and accept the usage conditions.

### Step 1: Authenticate with Hugging Face


In [1]:
from huggingface_hub import login
login()

### Step 2: Install dependencies

In [2]:
!pip install -q \
  fastapi \
  uvicorn \
  transformers \
  accelerate \
  bitsandbytes \
  pillow==10.4.0 \
  torch torchvision \





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.0 MB/s eta 0:00:00


## Step 3: Load MedGemma

In [3]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

MODEL_ID = "google/medgemma-4b-it"

processor = AutoProcessor.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()
print("✅ MedGemma loaded")


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

✅ MedGemma loaded


## Step 4: Install cloudflared

In [4]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

## Step 5: MEDICAL SYSTEM PROMPT

In [5]:
from fastapi import FastAPI, UploadFile, Form
from PIL import Image
import io
import json

app = FastAPI(title="ClinIQ – MedGemma API")

SYSTEM_PROMPT = """
You are a clinical decision support assistant.

Rules:
- Do NOT provide diagnoses
- Use observational language only
- Explicitly state uncertainty
- Phrase findings for clinicians
- Avoid prescriptive advice

Respond ONLY with valid JSON.

JSON schema:
{
  "observations": [],
  "possible_interpretations": [],
  "uncertainty_notes": "",
  "recommend_next_steps": []
}
"""

@app.post("/analyze")
async def analyze(
    image: UploadFile,
    clinical_question: str = Form(...)
):
    img_bytes = await image.read()
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

    full_prompt = f"""
Return ONLY valid JSON.

{SYSTEM_PROMPT}

Clinical question:
{clinical_question}
"""

    inputs = processor(
        images=img,
        text=full_prompt,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.4
        )

    text = processor.decode(output_ids[0], skip_special_tokens=True)

    # enforce JSON safety
    try:
        return json.loads(text)
    except Exception:
        return {
            "error": "Model did not return valid JSON",
            "raw_output": text
        }



## Step 6: Run FastAPI server



In [12]:
import logging
import uvicorn
from threading import Thread

# -----------------------
# Logging setup
# -----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s"
)

logger = logging.getLogger("cliniq")

def start_api():
    logger.info("Starting FastAPI server on 127.0.0.1:8000")

    uvicorn.run(
        app,
        host="127.0.0.1",
        port=8000,
        log_level="info",
        access_log=True
    )

    logger.info("Uvicorn process exited")

Thread(target=start_api).start()


INFO:     Started server process [542]


## Step 7 Expose via Cloudflare Tunnel

In [ ]:
import subprocess
import re

process = subprocess.Popen(
    [
        "./cloudflared-linux-amd64",
        "tunnel",
        "--no-autoupdate",
        "--protocol", "http2",        # ❌ no QUIC
        "--url", "http://127.0.0.1:8000"
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

for line in process.stdout:
    print(line, end="")
    if "trycloudflare.com" in line:
        print("\n🌍 COPY THIS URL ↑↑↑\n")


2026-02-05T11:50:05Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-05T11:50:05Z INF Requesting new quick Tunnel on trycloudflare.com...

🌍 COPY THIS URL ↑↑↑

2026-02-05T11:50:11Z INF +--------------------------------------------------------------------------------------------+
2026-02-05T11:50:11Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-05T11:50:11Z INF |  https://essentials-isolated-pub